In [ ]:
import pandas as pd
import os
from openai import OpenAI
import json

# This code requires an OpenAI API key. Before proceeding, you can create one on the OpenAI platform and add it to the relevant field in the creds.json file we have provided.
with open("creds.json") as js:
    api_key = json.load(js)['OPENAI_API_KEY']

client = OpenAI(api_key=api_key)

if not os.path.exists("predicted_labels/"):
    os.mkdir("predicted_labels/")

In [ ]:
prompt3="This statement may express a STANCE towards two politicians, Trump and Clinton. Stance represents the attitude expressed towards them. The stance options are Favor, Against or None. Provide the answer in the following format: {Trump: STANCE, Clinton: STANCE}\n\n"
test = pd.read_csv("data/fb_test.csv")
train_100 = pd.read_csv("data/fb_train_100.csv")

# Zero-shot

In [ ]:
generated_output = []
for comment in test.iterrows():
    message = prompt3 + comment[1]["prompt"]
    
    completion = client.chat.completions.create(
        model="gpt-4o",
        temperature=0,
        messages=[
            {"role": "user", "content": message}
            ]
        )
    generated_output.append(completion.choices[0].message.content)
    
test_preds = pd.concat([test, pd.DataFrame({"preds":generated_output})], axis=1)
test_preds.to_csv("predicted_labels/fb_gpt4o_zero.csv", index=False)

## Few-shot

In [ ]:
for i, eg in enumerate(train_100.iterrows()):
    generated_output = []
    for comment in test.iterrows():
        message = prompt3 + "###\n" + eg[1].iloc[0]+ "\n###\n" + comment[1]["prompt"]
    
        completion = client.chat.completions.create(
        model="gpt-4o",
        temperature=0,
            messages=[
                {"role": "user", "content": message}
                ]
            )
        print(completion.choices[0].message)
        generated_output.append(completion.choices[0].message.content)
        
    test_preds = pd.concat([test, pd.DataFrame({"preds":generated_output})], axis=1)
    test_preds.to_csv(f"predicted_labels/fb_gpt4o_one_eg_{i}.csv", index=False)